# Diabetes in 130 US hospitals for the years 1999 to 2008

## Checking algorithms

 * [Feature selection](#Feature-selection)
   - [Correlation feature selection](#Correlation-feature-selection)
   - [Recursive feature elimination](#Recursive-feature-elimination)
 * [Logistic regression](#Logistic-regression)
   - [Using statsmodels](#Using-statsmodels)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

In [3]:
df_encoded = pd.read_csv('data/df_encoded.csv', index_col=0)

In [4]:
df_encoded.head()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted,race=Caucasian,gender=Female,age=[10-20),max_glu_serum=None,A1Cresult=None,metformin=No,repaglinide=No,nateglinide=No,chlorpropamide=No,glimepiride=No,acetohexamide=No,glipizide=No,glyburide=No,tolbutamide=No,pioglitazone=No,rosiglitazone=No,acarbose=No,miglitol=No,troglitazone=No,tolazamide=No,examide=No,citoglipton=No,insulin=Up,glyburide-metformin=No,glipizide-metformin=No,glimepiride-pioglitazone=No,metformin-rosiglitazone=No,metformin-pioglitazone=No,change=Ch,diabetesMed=Yes,primary_diag=Others,secondary_diag=Diabetes,additional_diag=Others,race=AfricanAmerican,age=[20-30),glipizide=Steady,insulin=No,change=No,secondary_diag=Others,gender=Male,age=[30-40),additional_diag=Circulatory,age=[40-50),insulin=Steady,primary_diag=Neoplasms,secondary_diag=Neoplasms,age=[50-60),primary_diag=Circulatory,secondary_diag=Circulatory,age=[60-70),metformin=Steady,glimepiride=Steady,age=[70-80),glyburide=Steady,secondary_diag=Respiratory,age=[80-90),age=[90-100),rosiglitazone=Steady,additional_diag=Respiratory,primary_diag=Diabetes,additional_diag=Injury,glyburide=Up,additional_diag=Neoplasms,repaglinide=Up,insulin=Down,additional_diag=Diabetes,primary_diag=Respiratory,secondary_diag=Injury,additional_diag=Genitourinary,primary_diag=Injury,diabetesMed=No,secondary_diag=Musculoskeletal,race=Other,A1Cresult=>7,secondary_diag=Genitourinary,acarbose=Steady,primary_diag=Genitourinary,secondary_diag=Digestive,metformin=Up,additional_diag=Digestive,troglitazone=Steady,primary_diag=Musculoskeletal,primary_diag=Digestive,A1Cresult=>8,A1Cresult=Norm,repaglinide=Steady,additional_diag=Musculoskeletal,max_glu_serum=>300,glipizide=Up,max_glu_serum=Norm,max_glu_serum=>200,glipizide=Down,race=Asian,race=Hispanic,repaglinide=Down,age=[0-10),rosiglitazone=Up,glimepiride=Down,glimepiride=Up,tolazamide=Steady,pioglitazone=Steady,pioglitazone=Up,metformin=Down,glyburide=Down,tolbutamide=Steady,chlorpropamide=Steady,pioglitazone=Down,acarbose=Up,rosiglitazone=Down,glyburide-metformin=Steady,nateglinide=Steady,chlorpropamide=Down,chlorpropamide=Up,glyburide-metformin=Down,miglitol=Steady,glyburide-metformin=Up,nateglinide=Down,acetohexamide=Steady,miglitol=Down,nateglinide=Up,glipizide-metformin=Steady,miglitol=Up,metformin-pioglitazone=Steady
0,3,59,0,18,0,0,0,9,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,2,11,5,13,2,0,1,6,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,2,44,1,16,0,0,0,7,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,1,51,0,8,0,0,0,5,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.

In [5]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68629 entries, 0 to 68628
Columns: 132 entries, time_in_hospital to metformin-pioglitazone=Steady
dtypes: float64(123), int64(9)
memory usage: 69.6 MB


In [6]:
df_encoded.shape

(68629, 132)

# Feature selection

## Correlation

In [7]:
filter_ = df_encoded.corr()['readmitted'] < 1
corr = df_encoded.corr()['readmitted'][filter_].abs()
corr.sort_values(ascending=False, inplace=True)

In [8]:
print(corr[:20])

number_inpatient               0.140042
number_diagnoses               0.092481
number_emergency               0.074913
glipizide=Steady               0.065076
insulin=No                     0.065076
number_outpatient              0.061496
time_in_hospital               0.056052
acarbose=No                    0.044076
glimepiride-pioglitazone=No    0.043786
num_lab_procedures             0.043653
acetohexamide=No               0.040502
race=Asian                     0.039893
diabetesMed=Yes                0.038520
change=Ch                      0.038520
additional_diag=Injury         0.034193
metformin-rosiglitazone=No     0.031626
num_medications                0.030383
primary_diag=Diabetes          0.029723
num_procedures                 0.029285
glimepiride=Down               0.029238
Name: readmitted, dtype: float64


## Recursive feature elimination

We're going to perform [feature ranking with recursive feature elimination](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html), using the RFE class in scikit-learn. This procedure was taken from this blog post ['Building A Logistic Regression in Python, Step by Step'](https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8). As the blog post mentions, this technique "is based on the idea to repeatedly construct a model and choose either the best or worst performing feature, setting the feature aside and then repeating the process with the rest of the features. This process is applied until all features in the dataset are exhausted. The goal of RFE is to select features by recursively considering smaller and smaller sets of features."

In [9]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [10]:
X = df_encoded.drop('readmitted', axis=1)
y = df_encoded.loc[:, 'readmitted'].values.ravel()

The `readmitted` output column has a fairly similar amount of readmission and non-readmission cases.

In [11]:
print(f'Ratio of readmission to total: {(y == 1).sum()/y.shape[0]:.2f}')
print(f'Ratio of non-readmission to total: {(y == 0).sum()/y.shape[0]:.2f}')

Ratio of readmission to total: 0.40
Ratio of non-readmission to total: 0.60


In [12]:
from time import time
t0 = time()
lg = LogisticRegression(solver='newton-cg')
rfe = RFE(lg, 20)
rfe = rfe.fit(X, y)
print(f'Time elapsed: {time() - t0:.2f} sec.')

Time elapsed: 449.23 sec.


In [13]:
print(f'Number of reduced features: {rfe.n_features_}')
print(f'List of reduced features: {list(X.columns.values[rfe.support_])}')

Number of reduced features: 20
List of reduced features: ['number_inpatient', 'A1Cresult=None', 'nateglinide=No', 'chlorpropamide=No', 'troglitazone=No', 'examide=No', 'race=AfricanAmerican', 'secondary_diag=Neoplasms', 'glimepiride=Steady', 'age=[70-80)', 'glyburide=Steady', 'secondary_diag=Respiratory', 'acarbose=Steady', 'primary_diag=Musculoskeletal', 'primary_diag=Digestive', 'race=Asian', 'rosiglitazone=Up', 'tolbutamide=Steady', 'nateglinide=Up', 'metformin-pioglitazone=Steady']


In [14]:
X_mod = X.loc[:, list(X.columns.values[rfe.support_])]

In [15]:
print(X_mod.shape)
print(y.shape)

(68629, 20)
(68629,)


# Logistic regression

## Using statsmodels

In [16]:
import statsmodels.api as sm
logit_model = sm.Logit(y, X_mod)
result = logit_model.fit_regularized(method='l1')

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.6630438564590501
            Iterations: 185
            Function evaluations: 185
            Gradient evaluations: 185


In [17]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                68629
Model:                          Logit   Df Residuals:                    68609
Method:                           MLE   Df Model:                           19
Date:                Fri, 19 Jul 2019   Pseudo R-squ.:                 0.01733
Time:                        19:16:17   Log-Likelihood:                -45504.
converged:                       True   LL-Null:                       -46306.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
number_inpatient                  0.5188      0.015     33.882      0.000       0.489       0.549
A1Cresult=None                   -0.4725      0.144     -3.277      0.001 

In [18]:
print(result.summary2())

                                      Results: Logit
Model:                       Logit                    Pseudo R-squared:         0.017     
Dependent Variable:          y                        AIC:                      91048.0736
Date:                        2019-07-19 19:16         BIC:                      91230.8031
No. Observations:            68629                    Log-Likelihood:           -45504.   
Df Model:                    19                       LL-Null:                  -46306.   
Df Residuals:                68609                    LLR p-value:              0.0000    
Converged:                   1.0000                   Scale:                    1.0000    
No. Iterations:              185.0000                                                     
------------------------------------------------------------------------------------------
                               Coef.    Std.Err.     z     P>|z|     [0.025       0.975]  
-------------------------------------

There are 11 feature that have a p value greater than 0.05, and these are: `nateglinide=No`, `chlorpropamide=No`, `race=AfricanAmerican`, `secondary_diag=Neoplasms`, `glimepiride=Steady`, `age=[70-80)`, `glyburide=Steady`, `secondary_diag=Respiratory`, `primary_diag=Musculoskeletal`, `nateglinide=Up` and `metformin-pioglitazone=Steady`.

## Using scikit-learn

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split